# KNN algorithm 

library를 사용하지 않고 직접 프로그래밍하고 knn 알고리즘을 만들어보자. 

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# from sklearn.preprocessing import StandardScaler

< Follow this steps >

1. For each test data in “MNIST_test.csv”, compute distances with the training data.
2. Find the K-nearest neighbors and decide the majority class.
3. Compare the prediction with the ground truth in the test data\
a. Correctly classified if the predicted label and the ground truth is identical.\
b. Incorrectly classified if the predicted label and ground truth is NOT identical.
4. Repeat Step 1-4 for all data in the test data
5. Then, you can count how many test data are correctly classified and incorrectly classified.
6. Show the accuracy of your KNN. 

In [85]:
df_train = pd.read_csv("http://mkang.faculty.unlv.edu/teaching/CS489_689/HW2/MNIST_training.csv")
df_test = pd.read_csv("http://mkang.faculty.unlv.edu/teaching/CS489_689/HW2/MNIST_test.csv") 

In [86]:
df_train.shape

(949, 785)

In [87]:
df_test.shape

(50, 785)

In [88]:
train_x = df_train.iloc[:, 1:]
train_y = df_train.iloc[:, 0]
test_x = df_test.iloc[:, 1:]
test_y = df_test.iloc[:, 0]

In [89]:
train_y

0      0
1      0
2      0
3      0
4      0
      ..
944    9
945    9
946    9
947    9
948    9
Name: label, Length: 949, dtype: int64

### 1. compute distances 

유클리디안 거리를 이용하여 거리를 구하기 (유클리디안 거리 함수는 파이썬 라이브러리를 활용함. )

In [90]:
from sklearn.metrics import pairwise_distances

In [91]:
distance = pairwise_distances(test_x, train_x)  

In [92]:
sorted_distance = np.argsort(distance)
sorted_distance

array([[ 83,  89,  37, ..., 808, 926, 899],
       [ 34,  33,  21, ..., 663, 619, 662],
       [ 73, 273,  79, ..., 724, 363,  52],
       ...,
       [940, 879, 931, ...,  65,  34,  33],
       [947, 912, 936, ...,  52,  34,  33],
       [885, 940, 943, ...,  59,  34,  33]], dtype=int64)

### 2. Find the K-nearest neighbors & decide the classes 

In [93]:
k = 5   # change the k ( only for odd ) 

In [94]:
topk_distance = sorted_distance[:, :k]
topk_distance

array([[ 83,  89,  37,  34,  85],
       [ 34,  33,  21,   5,  28],
       [ 73, 273,  79,   6, 854],
       [ 13,  15,  41,  44,  81],
       [ 94,  21,  28,  33,  34],
       [181, 117, 168, 159, 150],
       [186, 163, 183,  97, 172],
       [187, 141,  97, 176, 103],
       [105, 147, 121, 131, 111],
       [128, 184, 169, 122, 125],
       [299, 847, 809, 211, 767],
       [206, 279, 215, 212, 251],
       [206, 212, 279, 325, 137],
       [703, 648, 625, 612, 733],
       [269, 190, 703, 214, 205],
       [710, 759, 348, 859, 898],
       [376, 200, 309, 320, 304],
       [299, 334, 319, 377, 842],
       [339, 305, 308, 360, 371],
       [291, 314, 376, 304, 308],
       [444, 205, 416, 190, 759],
       [441, 460, 422, 404, 425],
       [461, 445, 413, 384, 380],
       [421, 468, 381, 410, 383],
       [915, 923, 673, 878, 909],
       [554, 538, 537, 536, 883],
       [613, 618, 810, 476, 642],
       [476, 491, 807, 543, 810],
       [542, 569, 551, 718, 505],
       [546, 4

In [95]:
import statistics
pred=[]

for idx in topk_distance:
    # print(train_y[ids-x])
    #print(statistics.mode(train_y[idx]))
    pred.append(statistics.mode(train_y[idx]))

In [96]:
pred   # prediction of group ( knn ) 

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 8,
 2,
 2,
 6,
 2,
 9,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 9,
 5,
 6,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 9,
 7,
 9,
 9,
 9]

### 4. Accuracy

In [97]:
acc = (test_y == pred).sum() / len(test_y)
acc

0.88

# 2023.03.30 

# Cross-validation 

### K-fold CV

k-fold의 경우, k=5~10를 자주 사용하고, 실제 프로젝트에서는 데이터 레이블의 분포가 균등하지는 않기 때문에 Stratified cross-validation을 자주 사용한다. 

*현재의 MNIST 데이터의 경우 레이블 당 해당하는 데이터가 균등하기 때문에 큰 효과는 없음. 


In [98]:
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

앞서 사용했던 train data와 test data를 하나로 합친다. 

In [99]:
# create dataset

df = pd.concat([df_train, df_test])

In [100]:
# 데이터 분할 

X = df.iloc[:,1:]
y = df['label']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.3)

In [101]:
# KNN 알고리즘 적합 

def Euclidean_dist(pt1, pt2):
    distance = np.sqrt(np.sum((pt1 - pt2) ** 2))
    return distance

def Nearest_neighbors(train, test_obs, n):
    neighbor_distance = []
    for i in range(len(train)):
        dist = Euclidean_dist(train[i, 1:], test_obs[1:])  
        neighbor_distance.append((dist, train[i, 0])) 
    neighbor_distance.sort(key=lambda x: x[0])
    nearest_neighbors = neighbor_distance[:n]
    labels = [neighbor[1] for neighbor in nearest_neighbors]
    prediction = max(labels, key=labels.count)
    return prediction

# 데이터프레임을 NumPy 배열로 변환
train_array = df_train.values
test_obs_array = df_test.iloc[1, 1:].values
test_label = df_test.iloc[1, 0]
test_array = np.concatenate((test_obs_array, [test_label]))

neighbor = []

for i in range(0, 50):
    test_obs_array = df_test.iloc[i, 1:].values
    test_label = df_test.iloc[i, 0]
    test_array = np.concatenate((test_obs_array, [test_label]))
    neighbor.append(Nearest_neighbors(train_array, test_array, 3)) # k=9

In [102]:
kfold = KFold(n_splits=10) 
n_iteration = 0 
for train_index, test_index in kfold.split(df):
    n_iteration += 1
    label_train = df['label'].iloc[train_index]
    label_test = df['label'].iloc[test_index]

In [103]:
print(label_train.value_counts())

0    95
1    95
2    95
3    95
4    95
5    95
7    95
8    95
6    94
9    46
Name: label, dtype: int64


In [104]:
print(label_test.value_counts())

9    54
0     5
1     5
2     5
3     5
4     5
5     5
6     5
7     5
8     5
Name: label, dtype: int64
